<a href="https://colab.research.google.com/github/syibrahima31/Project-with-torch/blob/main/Logistic_Regression_with_Titanic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [173]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot  as plt 
import torch 
import torch.nn as nn 
from torch.utils.data import DataLoader, Dataset
from seaborn import load_dataset
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import torch.optim as optim 

## PREPARE THE DATASET 

In [174]:
titanic = load_dataset("titanic").select_dtypes(["int64","float64"])

# delete  Missing values 
titanic = titanic.dropna()


X, y = titanic.iloc[:, 1:].values, titanic.iloc[:, 0].values
y = y.reshape(-1,1)
# Normalise the dataset 

scaler =StandardScaler()
X = scaler.fit_transform(X)

In [175]:
class ToTensor:
  def __call__(self, sample):
    inputs, labels = sample 
    return torch.from_numpy(inputs), torch.from_numpy(labels)

In [176]:
class TitDataset(Dataset): 
  def __init__(self, X, y, transform= None):
    self.X_train  = X.astype(np.float32)
    self.y_train  = y.astype(np.float32)
    self.n_sample = len(self.X_train)
    self.transform = transform 
    


  def __getitem__(self, index):
    sample = self.X_train[index], self.y_train[index]

    if self.transform : 
      sample = self.transform(sample)

    return sample

  def __len__(self):
    return   self.n_sample

In [177]:
# use Dataloader 
dataset = TitDataset(X, y , transform=ToTensor())
batch_size = 50
dataloader = DataLoader(dataset=dataset, batch_size=batch_size, shuffle=True)

n_features = X.shape[1]

## CREATE LOGISTIC REGRESSION MODEL 

In [196]:
 class LogisticRegression(nn.Module):
   def __init__(self, in_features, out_features=1):
     super(LogisticRegression, self).__init__()
     self.layer = nn.Linear(in_features, out_features)

   def forward(self, x):
    x = self.layer(x)
    return torch.sigmoid(x)    

In [197]:
model = LogisticRegression(n_features)

## DEFINE LOSS AND OPTIMIZER 

In [198]:
learning_rate = 0.01
criterion = nn.BCELoss()
optimizer = optim.SGD(model.parameters(), lr = learning_rate)

## TRAINING LOOP 

In [212]:
n_epochs = 20
n_iterations = len(dataloader)
L = []

for epoch in range(n_epochs):
  
  for i,(data, target) in enumerate(dataloader):
    # Forward pass 
    prediction = model(data)
   
    # loss function 
    loss = criterion(prediction, target)

    # compute the gradient
    loss.backward()

    # update the weight 
    optimizer.step()

    # zero gradient 
    optimizer.zero_grad()

    if (i+ 1) % 10 ==0:
      print(f"epochs:{epoch}/{n_epochs}, steps:{i}/{n_iterations}, loss={loss.item()}")

    with torch.no_grad():
      pred_probs  = model(data)
      pred_labels = pred_probs.round()
      acc = pred_labels.eq(target).sum() / data.shape[0]
      L.append(acc.item())




epochs:0/20, steps:9/15, loss=0.6236314177513123
epochs:1/20, steps:9/15, loss=0.67609041929245
epochs:2/20, steps:9/15, loss=0.5678597092628479
epochs:3/20, steps:9/15, loss=0.5792040824890137
epochs:4/20, steps:9/15, loss=0.5958834290504456
epochs:5/20, steps:9/15, loss=0.4794982075691223
epochs:6/20, steps:9/15, loss=0.5449826717376709
epochs:7/20, steps:9/15, loss=0.5852930545806885
epochs:8/20, steps:9/15, loss=0.5348368287086487
epochs:9/20, steps:9/15, loss=0.5403425097465515
epochs:10/20, steps:9/15, loss=0.6452415585517883
epochs:11/20, steps:9/15, loss=0.5748589038848877
epochs:12/20, steps:9/15, loss=0.49865615367889404
epochs:13/20, steps:9/15, loss=0.598423421382904
epochs:14/20, steps:9/15, loss=0.5951711535453796
epochs:15/20, steps:9/15, loss=0.5542836785316467
epochs:16/20, steps:9/15, loss=0.5359903573989868
epochs:17/20, steps:9/15, loss=0.6309725046157837
epochs:18/20, steps:9/15, loss=0.47078341245651245
epochs:19/20, steps:9/15, loss=0.5635089874267578
